In [78]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn import metrics

col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age']
df = pd.read_csv('diabetes.csv',header=None, names=col_names)
df = df.iloc[1:]
df.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1


In [37]:
label = df['label']
features = df.drop('label',axis=1)

In [38]:
features.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age
1,6,148,72,35,0,33.6,0.627,50
2,1,85,66,29,0,26.6,0.351,31
3,8,183,64,0,0,23.3,0.672,32
4,1,89,66,23,94,28.1,0.167,21
5,0,137,40,35,168,43.1,2.288,33


In [40]:
X_train,X_test, y_train, y_test = train_test_split(features,label, test_size=0.3, random_state=1)

In [42]:
classifier = DecisionTreeClassifier()
model = classifier.fit(X_train,y_train)

In [43]:
y_pred = model.predict(X_test)

In [45]:
accuracy = metrics.accuracy_score(y_test,y_pred)
accuracy*100

68.83116883116884

# Lets optimize

In [69]:
from sklearn.model_selection import GridSearchCV,ShuffleSplit
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score

def performance_metric(y_true,y_pred):
    score = r2_score(y_true,y_pred)
    return score

def model(x,y):
    
    classifier = DecisionTreeClassifier(criterion="entropy")
    
    cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=0)
    parameters = {'max_depth':np.arange(1,11), 'min_samples_leaf':np.arange(1,11)}
    scorer = make_scorer(performance_metric)
    
    grid = GridSearchCV(classifier, parameters, scoring=scorer, cv=cv)
    grid = grid.fit(x,y)
    
    return grid.best_estimator_

best_model = model(X_train,y_train)

In [70]:
best_model.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 10,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [72]:
y_predicted = best_model.predict(X_test)
accuracy2 = metrics.accuracy_score(y_test,y_predicted)
accuracy2*100

78.35497835497836

## Hmm, 68% to 78% Nicee

In [73]:
import pickle

diabetes_model = pickle.dumps(best_model)